In [ ]:
import pathlib
import pandas as pd

datadir = pathlib.Path.cwd().parent / "data"

pd_hotel       = pd.read_parquet(datadir / "hotel.parquet")
pd_reservation = pd.read_parquet(datadir / "reservation.parquet")

# 14章 ウィンドウ関数
## 14-1 順序のないウィンドウ関数
### Q: ホテルマスタに対してホテルのある市区町村の平均価格の付与


#### Not Awesome

In [ ]:
(
    pd_hotel
    #（2）pd_hotelに（1）の結果をleft join
    .merge(
        pd_hotel
        #（1）address_prefectureとaddress_cityごとにunit_priceの平均を算出
        .groupby(["address_prefecture", "address_city"]).unit_price.mean()
        .rename("avg_price_within_city"),
        on=["address_prefecture", "address_city"], how="left"
    )
)

#### Awesome

In [ ]:
pd_hotel.assign(avg_price_within_city=lambda df:
    # address_prefectureとaddress_cityごとにunit_priceの平均を算出
    # transform関数を用いているので、行数はgroupby処理前と同じになる
    df.groupby(["address_prefecture", "address_city"]).unit_price.transform("mean")
)

### Q: 都道府県ごとのホテル数の構成比の算出


#### Awesome

In [ ]:
(
    pd_hotel
    #（1）都道府県ごとのホテル数を計算
    .groupby("address_prefecture").size().rename("hotel_cnt")
    .to_frame()
    #（2）都道府県ごとのホテル数を、都道府県ごとのホテル数の合計（=全ホテル数）で除算して構成比を算出
    .assign(ratio=lambda df: df.hotel_cnt / df.hotel_cnt.sum())
)

## 14-2 順序のあるウィンドウ関数
### Q: 各顧客の一つ前と二つ前の予約の予約金額の取得


#### Awesome

In [ ]:
(
    pd_reservation
    #（1）reserved_atの昇順（時系列順）にソート
    .sort_values("reserved_at")
    .assign(
        #（2）customer_idごとに一つ前と二つ前の行のtotal_priceを取得
        total_price_at_prev=lambda df: df.groupby("customer_id").total_price.shift(1),
        total_price_at_2prev=lambda df: df.groupby("customer_id").total_price.shift(2)
    )
)

### Q: 顧客ごとに予約順を付与


#### Awesome

In [ ]:
pd_reservation.assign(reservation_no=lambda df: 
    df.groupby("customer_id").reserved_at.rank(method="first"))

## 14-3 範囲のあるウィンドウ関数
### Q: 売上の移動平均の算出


#### Awesome

In [ ]:
(
    pd_reservation
    #（1）キャンセルを除去
    .query("status != 'canceled'")
    #（2）checkout_dateを月周期のPeriodに変換
    .assign(month=lambda df: df.checkout_date.dt.to_period("M"))
    #（3）monthごとにtotal_priceの総和を計算（集計結果はmonth順に並ぶ）
    .groupby("month").total_price.sum().rename("sales")
    .to_frame()
    #（4）salesの直近3行分の平均を計算
    .assign(moving_avarage_sales=lambda df: df.sales.rolling(3).mean())
)

### Q: 顧客の過去の総予約金額を付与


#### Awesome

In [ ]:
(
    pd_reservation
    #（1）キャンセル済みのデータを除去
    .query("status != 'canceled'")
    #（2）reserved_atの昇順（時系列順）にソート
    .sort_values("reserved_at")
    .assign(
        #（3）customer_idごとにtotal_priceを時系列順に1行分ずらす
        # ある行には、元々一つ前の行だった行の値がくる
        prev_total_price=lambda df: df.groupby("customer_id").total_price.shift(1),
        #（4）customer_idごとにprev_total_priceの累計和を計算
        past_total_spent=lambda df: df.groupby("customer_id").prev_total_price.cumsum()
    )
    #（5）一時的に作成したprev_total_price列を削除
    .drop(columns="prev_total_price")
)

In [ ]:
(
    pd_reservation
    #（1）キャンセル済みのデータを除去
    .query("status != 'canceled'")
    #（2）reserved_atの昇順（時系列順）にソート
    .sort_values("reserved_at")
    #（3）customer_idごとにtotal_priceの累計和を計算し、現在の行の値を減算
    .assign(past_total_spent=lambda df:
        df.groupby("customer_id").total_price.cumsum() - df.total_price)
)

### Q: 顧客の過去90日間の総予約金額を付与


#### Awesome

In [ ]:
(
    pd_reservation
    #（1）キャンセル済みのデータを除去
    .query("status != 'canceled'")
    #（2）reserved_atの昇順（時系列順）にソート
    .sort_values("reserved_at")
    #（6）total_spent_last_90days列に（3）〜（5）の結果を格納
    .assign(total_spent_last_90days=lambda df:
        df
        #（3）reserved_atをIndexに設定
        .set_index("reserved_at")
        #（4）customer_idごとに過去90日間のtotal_priceの総和を算出
        .groupby("customer_id")
        .total_price.transform(lambda s: s.rolling("90D", closed="left").sum())
        #（5）Indexを消して元のDataFrameに格納するため、ndarrayを取得
        .values
    )
)

## 14-4 指定列が最小／最大となる行の取得
### Q：顧客の初回予約時の情報の取得


#### Not Awesome

In [ ]:
(
    pd_reservation
    [["customer_id", "reserved_at", "hotel_id"]]
    # customer_idごとにreserved_atの最小値をとる行のIndexを取得し
    # loc関数でその行を抽出
    .loc[lambda df: df.groupby("customer_id").reserved_at.idxmin()]
)

#### Awesome

In [ ]:
(
    pd_reservation
    # （1）顧客ごとに予約順の番号を付与
    .assign(rn=lambda df: df.groupby("customer_id").reserved_at.rank(method="first"))
    # （2）顧客の最初の予約のみ抽出
    .query("rn == 1")
    [["customer_id", "reserved_at", "hotel_id"]]
)

## 14-5 層別サンプリング
### Q: 都道府県別に層別サンプリング


#### Not Awesome

In [ ]:
# 10%のデータをランダムサンプリング
pd_hotel.sample(frac=0.1)

#### Awesome

In [ ]:
# address_prefecture列の値ごとに10%のデータをランダムサンプリング
pd_hotel.groupby("address_prefecture").sample(frac=0.1)

## 14-6 グループ単位のランダムサンプリング
### Q: 顧客単位で予約履歴をランダムサンプリング


#### Not Awesome

In [ ]:
# 20000件をランダムサンプリング
pd_reservation.sample(20000)

#### Awesome

In [ ]:
(
    pd_reservation
    # （2）判定結果を用いて予約を抽出
    .loc[lambda df:
        # （1）顧客をサンプリングし、サンプリングした顧客の予約を判定
        df.customer_id.isin(pd.Series(df.customer_id.unique()).sample(frac=0.01))
    ]
)